# Pathway similarity network

#### This notebooks generate the file adapted to [Cytoscape](http://www.cytoscape.org/)  to plot the pathway similarity network

Author: [Daniel Domingo-Fernández](https://github.com/ddomingof) 



In [1]:
import time
import sys
import os

import networkx as nx
import pandas as pd
from collections import defaultdict
import itertools as itt
import json

In [2]:
time.asctime()

'Wed Feb 21 16:09:59 2018'

In [3]:
print(sys.version)

3.4.5 (default, Dec 11 2017, 14:22:24) 
[GCC 4.8.5 20150623 (Red Hat 4.8.5-16)]


In [4]:
#Location of the cloned repo
BASE_PATH = os.environ['COMPATH']
EXCELS_PATH = os.path.join(BASE_PATH,'src','compath','static','resources','excel')


In [5]:
kegg_excel = os.path.join(EXCELS_PATH,'kegg_gene_sets.csv')
reactome_excel= os.path.join(EXCELS_PATH,'reactome_gene_sets.csv')
wikipathways_excel = os.path.join(EXCELS_PATH,'wikipathways_gene_sets.csv')

In [6]:
def create_pathway_gene_set_dict(dataframe):
    """Creates a pathway genes dictionary
    
    :param pandas.DataFrame dataset: gene sets df
    :rtype: collections.defaultdict
    :returns: dictionary of pathway gene sets
    """
    
    pathway_dictionary = defaultdict(set)
    
    for pathway_name in dataframe: # iterate over columns in dataframe

        for gene in dataframe[pathway_name].unique():
            if not isinstance(gene, str): # There are NaN in the Pandas nArray
                continue

            pathway_dictionary[pathway_name].add(gene)
            
    return pathway_dictionary

Load KEGG

In [7]:
kegg_dataframe = pd.read_csv(kegg_excel, dtype=object)

# Remove the 'Homo sapiens' out of the KEGG pathways
kegg_dataframe.columns = [
    kegg_pathway.replace(' - Homo sapiens (human)', '')
    for kegg_pathway in kegg_dataframe
] 

kegg_pathways = create_pathway_gene_set_dict(kegg_dataframe)
        
assert (len(kegg_pathways.keys()) == 323)

Load Reactome

In [8]:
reactome_dataframe = pd.read_csv(reactome_excel, dtype=object)

reactome_pathways = create_pathway_gene_set_dict(reactome_dataframe)

assert (len(reactome_pathways.keys()) == 2132) # Total of 2636 of those: 2132 are not empty

Load WikiPathways

In [9]:
wikipathways_dataframe = pd.read_csv(wikipathways_excel, dtype=object)

wikipathways_pathways = create_pathway_gene_set_dict(wikipathways_dataframe)

assert (len(wikipathways_pathways.keys()) == 408)

Create network

In [21]:
def export_cytoscape_js(graph):
    """Convert networkx to Cytoscape.js JSON format
    
    :param: networkx.Graph graph: graph
    :rtype: dict
    :return: dictionary representing the json object in cytoscape js
    """    
    network_dict = defaultdict(list)
    
    info_to_id = {} # linking pathway info tuple to identifier
    
    for node_id, node in enumerate(graph.nodes()):
        
        info_to_id[node] = node_id
        
        node_object = {}
        node_object["data"] = {}
        node_object["data"]["id"] = node_id
        node_object["data"]["name"] = node[0]
        node_object["data"]["resource"] = node[1]
        # TODO: Add link identifier reactome/kegg/wikipathways
        network_dict["nodes"].append(node_object.copy())
        

    for source, target, data in graph.edges(data=True):
        
        nx = {}
        nx["data"]={}
        nx["data"]["source"]=info_to_id[source]
        nx["data"]["target"]=info_to_id[target]
        nx["data"]["similarity"]=data['similarity']
        
        network_dict["edges"].append(nx)
        
    return network_dict


def populate_similarity_network(graph, threshold, pathway_resource_1, gene_sets_1, pathway_resource_2, gene_sets_2):
    """Populates the similarity pathway network in place given two gene sets a threshold
    
    :param networkx.Graph graph: similarity pathway network
    :param float threshold: percentage of the minimum threshold to add the link to the network (e.g., 0.75 means 75% similarity)
    :param str pathway_resource_1: name of the resource 1
    :param dict gene_sets_1: pathway gene set dict 2
    :param str pathway_resource_2: name of the resource 2
    :param dict gene_sets_2: pathway gene set dict 2
    """
    
    for pathway_1, pathway_2 in itt.product(gene_sets_1.keys(), gene_sets_2.keys()):
        
        if (pathway_1,pathway_resource_1) == (pathway_2,pathway_resource_2):
            continue

        intersection = len(gene_sets_1[pathway_1].intersection(gene_sets_2[pathway_2]))
        smaller_set = min(len(gene_sets_1[pathway_1]), len(gene_sets_2[pathway_2]))

        similarity = float(intersection/smaller_set) # Formula to calculate similarity

        if similarity < threshold: 
            continue
        
        graph.add_edge((pathway_1,pathway_resource_1),(pathway_2,pathway_resource_2), similarity=similarity)
        
        
    return graph



In [22]:
pathway_similarity_network = nx.Graph()

threshold = 0.90
KEGG = 'KEGG'
REACTOME = 'Reactome'
WIKIPATHWAYS = 'WikiPathways'

# Populate KEGG vs KEGG
pathway_similarity_network = populate_similarity_network(
    pathway_similarity_network, 
    threshold,
    KEGG, kegg_pathways,
    KEGG, kegg_pathways
)



json_data = export_cytoscape_js(pathway_similarity_network)


In [20]:
json.dumps(json_data)

'{"edges": [{"data": {"source": 0, "target": 7, "id": 0, "similarity": 0.9795918367346939}}, {"data": {"source": 1, "target": 54, "id": 1, "similarity": 0.9736842105263158}}, {"data": {"source": 2, "target": 24, "id": 2, "similarity": 0.9310344827586207}}, {"data": {"source": 2, "target": 7, "id": 3, "similarity": 1.0}}, {"data": {"source": 3, "target": 7, "id": 4, "similarity": 0.925}}, {"data": {"source": 4, "target": 26, "id": 5, "similarity": 1.0}}, {"data": {"source": 4, "target": 53, "id": 6, "similarity": 0.9}}, {"data": {"source": 4, "target": 7, "id": 7, "similarity": 0.9741379310344828}}, {"data": {"source": 5, "target": 14, "id": 8, "similarity": 1.0}}, {"data": {"source": 5, "target": 7, "id": 9, "similarity": 1.0}}, {"data": {"source": 6, "target": 7, "id": 10, "similarity": 0.9629629629629629}}, {"data": {"source": 7, "target": 64, "id": 11, "similarity": 1.0}}, {"data": {"source": 7, "target": 8, "id": 12, "similarity": 0.96}}, {"data": {"source": 7, "target": 11, "id": 